# Basics of Quantum Neural Network Structure

This notebook is useful to understand the basics of the quantum neural network. To get hands on with instantizatio of Estimator QNNs, and Sampler QNNs.

In [ ]:
from qiskit.utils import algorithm_globals
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR
from qiskit.algorithms.optimizers import L_BFGS_B
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
algorithm_globals.random_seed = 42
from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes

## Estimator QNN

Estimator QNN takes as input a parameterised circuit. To constructor a Estimator QNN first we need to construct a quantum circuit which will act as input.

Creating a quantum cicuit.

In [ ]:
from qiskit.circuit import Parameter
from qiskit import QuantumCircuit

params1 = [Parameter("input1"), Parameter("weight1")]
qc1 = QuantumCircuit(1)
qc1.h(0)
qc1.ry(params1[0], 0)
qc1.rx(params1[1], 0)
qc1.draw("mpl")


We will now create a quantum observable in the form of Y*no_of_qbits of circuit.

In [ ]:
from qiskit.quantum_info import SparsePauliOp

observable1 = SparsePauliOp.from_list([("Y" * qc1.num_qubits, 1)])

Estimator QNN takes the following input:

1. Quantum circuit - qc1 constructed in this case
2. Quantum observable - observable1 based on the Y*no_of_qbit
3. estimator - (optional) primitive instance
4. input_params - list of quantum circuit parameters that can be treated as network inputs.
5. weight_params - list of quantum circuit parameters that can be treated as weight parameter.


Lets create a Estimator QNN

In [ ]:
from qiskit_machine_learning.neural_networks import EstimatorQNN

estimator_qnn = EstimatorQNN(
    circuit=qc1, observables=observable1, input_params=[params1[0]], weight_params=[params1[1]]
)
estimator_qnn

## Sampler QNN

Sampler QNN is constructed in a similar way to Estimator QNN but it does not require a custom observable.

Lets create a Sampler QNN with 2 input parameters and 4 trainable weights.

In [ ]:
# construct simple feature map
param_x = Parameter("x")
feature_map = QuantumCircuit(1, name="fm")
feature_map.ry(param_x, 0)

# construct simple ansatz
param_y = Parameter("y")
ansatz = QuantumCircuit(1, name="vf")
ansatz.ry(param_y, 0)

# construct a circuit
qc = QuantumCircuit(1)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)




In [ ]:
# Creating two qubit circuit
num_qubits = 2
feature_map = ZZFeatureMap(feature_dimension=num_qubits)
ansatz = RealAmplitudes(num_qubits=num_qubits, reps=1)

qc = QuantumCircuit(num_qubits)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)

In [ ]:
qc.draw("mpl")

In [ ]:
def parity(x):
    return "{:b}".format(x).count("1") % 2

In [ ]:
from qiskit_machine_learning.neural_networks import SamplerQNN

sampler_qnn = SamplerQNN(circuit=qc, input_params=feature_map.parameters, weight_params=ansatz.parameters,interpret=parity , output_shape=2)
sampler_qnn

## VQR Regression

In [ ]:
vqr = VQR(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=L_BFGS_B(maxiter=5),
    callback=callback_graph,
)

## Linear Regression using SamplerQNN

In [ ]:
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

In [ ]:
# construct the regressor from the neural network
regressor = NeuralNetworkRegressor(
    neural_network=sampler_qnn,
    loss="squared_error",
    optimizer=L_BFGS_B(maxiter=30),
    callback=callback_graph,
)

### Sample Data

In [ ]:
num_samples = 50
eps = 0.2
lb, ub = -np.pi, np.pi
X_ = np.linspace(lb, ub, num=50).reshape(50, 1)
f = lambda x: np.sin(x) 

X = (ub - lb) * algorithm_globals.random.random([num_samples, 1]) + lb
y = f(X[:, 0]) + eps * (2 * algorithm_globals.random.random(num_samples) - 1)

plt.plot(X_, f(X_), "r--")
plt.plot(X, y, "bo")
plt.show()

In [ ]:
y.reshape(-1,1).shape

In [ ]:
x_new = y.reshape(-1,1)

In [ ]:
X.reshape(-1)

In [ ]:
y_new = X.reshape(-1)

In [ ]:
# create empty array for callback to store evaluations of the objective function
objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)

# fit to data
regressor.fit(x_new, y_new)

# # return to default figsize
plt.rcParams["figure.figsize"] = (6, 4)

# # score the result
regressor.score(x_new, y_new)

In [ ]:
y_pred = regressor.predict(x_new)

In [ ]:
y_pred.shape

In [ ]:
y_pred